In [1]:
import torch

class CustomReLU(torch.nn.Module):
    """
    Custom implementation of the ReLU activation function.
    """
    def __init__(self):
        super(CustomReLU, self).__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Applies the ReLU function element-wise: max(0, x).
        Args:
            x (torch.Tensor): Input tensor
        Returns:
            torch.Tensor: Output tensor with ReALU applied
        """
        return torch.maximum(x, torch.zeros_like(x))

In [2]:
import torch

class ConvolutionalLayer:
    """
    A utility class to create a convolutional layer with a custom ReLU activation.
    """
    @staticmethod
    def create(in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int) -> torch.nn.Sequential:
        """
        Creates a convolutional layer with a custom ReLU activation.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            kernel_size (int): Size of the convolution kernel
            stride (int): Stride of the convolution
            padding (int): Padding added to the input
        Returns:
            torch.nn.Sequential: A sequential layer containing Conv2d and CustomReLU
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            CustomReLU()
        )

In [3]:
import torch

class FullyConnectedLayer:
    """
    A utility class to create Fully Connected (FC) layers with Xavier initialization.
    """
    @staticmethod
    def Dense(input_dim: int, output_dim: int) -> torch.nn.Linear:
        """
        Creates an FC layer and applies Xavier initialization.
        Args:
            input_dim (int): Number of input features
            output_dim (int): Number of output features
        Returns:
            torch.nn.Linear: Initialized Fully-Connected layer
        """
        layer = torch.nn.Linear(input_dim, output_dim, bias=True)
        torch.nn.init.xavier_uniform_(layer.weight)
        return layer

In [4]:
import torch

class CustomAlexNet(torch.nn.Module):
    """
    Implementation of the AlexNet model.
    Input: Image tensor (batch_size, 3, 227, 227)
    Output: Class scores (batch_size, 1000)
    """
    def __init__(self, dropout_rate=0.5):
        """
        Args:
            dropout_rate (float): Dropout probability (default=0.5)
        """
        super(CustomAlexNet, self).__init__()

        # Convolutional and pooling layers
        self.layer1 = torch.nn.Sequential(
            ConvolutionalLayer.create(3, 96, 11, 4, 0),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.layer2 = torch.nn.Sequential(
            ConvolutionalLayer.create(96, 256, 5, 1, 2),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.layer3 = ConvolutionalLayer.create(256, 384, 3, 1, 1)
        self.layer4 = ConvolutionalLayer.create(384, 384, 3, 1, 1)

        self.layer5 = torch.nn.Sequential(
            ConvolutionalLayer.create(384, 256, 3, 1, 1),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        # Fully Connected layers and dropout
        self.layer_drop = torch.nn.Dropout(p=dropout_rate)

        self.layer6 = FullyConnectedLayer.Dense(6 * 6 * 256, 4096)

        self.layer7 = FullyConnectedLayer.Dense(4096, 4096)

        self.layer8 = FullyConnectedLayer.Dense(4096, 1000)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass of the model.
        Args:
            x (torch.Tensor): Input image tensor (batch_size, 3, 227, 227)
        Returns:
            torch.Tensor: Class scores (batch_size, 1000)
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer_drop(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        return x

In [5]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torchsummary import summary

# Check device compatibility (GPU or CPU)
def get_device():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device_name = "GPU" if torch.cuda.is_available() else "CPU"
    print(f"Using {device_name} for computation.")
    return device

# CIFAR-10 Dataset Load
def get_dataloader(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize((227, 227)),  # AlexNet의 입력 크기 227x227
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Imagenet 평균과 표준편차
    ])

    # Training DataLoader
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

    # Test DataLoader
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

    return trainloader, testloader

# Print Model Summary
def print_model_summary(model, device):
    summary(model.to(device), input_size=(3, 227, 227))

# Model Trainning
def train(model, trainloader, criterion, optimizer, device, epochs=1, testloader=None):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(trainloader)
        train_accuracy = correct / total * 100

        # If testloader is provided, calculate test loss
        if testloader:
            model.eval()
            test_running_loss = 0.0
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    test_running_loss += loss.item()
            test_loss = test_running_loss / len(testloader)
        else:
            test_loss = "N/A"  # Default value if no testloader is provided

        # Print metrics
        print(f"Epoch [{epoch + 1}/{epochs}] - "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
              f"Test Loss: {test_loss}")

# Model Test
def test(model, testloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# Trainning & Evaluation
def main(epochs=10):
    # Set device
    device = get_device()

    trainloader, testloader = get_dataloader()

    model = CustomAlexNet()
    print_model_summary(model, device)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train(model, trainloader, criterion, optimizer, device, epochs)
    test(model, testloader, device)

if __name__ == "__main__":
    epochs = 100  # Set the number of epochs for training
    main(epochs)

Using GPU for computation.


100%|██████████| 170498071/170498071 [00:02<00:00, 76839469.82it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
        CustomReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
        CustomReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
        CustomReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
       CustomReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
       CustomReLU-12          [-1, 256, 13, 13]               0
        MaxPoo